In [1]:
# plot the audio data
# use popup window
%matplotlib qt
import matplotlib.pyplot as plt

from utils import *# plot the audio data
# use popup window
%matplotlib qt
import matplotlib.pyplot as plt

from utils import *

chrip freq 5000, 15000.0
chrip freq 7000, 11000.0


In [2]:
rate = 48000
symbol_samples = 48
t_symbol = symbol_samples / rate
delta_f = 1 / t_symbol

print(f"Length of a symbol: {t_symbol} s")
print(f"Frequency resolution: {delta_f} Hz")

Length of a symbol: 0.001 s
Frequency resolution: 1000.0 Hz


In [135]:
channels = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
channel_weight = {channel: 1 / len(channels) for channel in channels}

def gen_symbol(n_freqs):
    fft_freq = np.fft.rfftfreq(symbol_samples, 1 / rate)
    fft_amp = np.zeros(fft_freq.shape, dtype=np.complex128)

    # amp = [0, 0.52179533, 0.30938262, 0.27554025, 0.1861606, 0.21071286]
    # amp_corr = {channel: 1 for channel in channels}
    # 1kHz-5kHz
    # amp_corr = {1: np.float32(2.5360713), 2: np.float32(1.2784102), 3: np.float32(1.1017478), 4: np.float32(1.1441331), 5: np.float32(1.11869)}
    
    # 2kHz-12kHz
    # amp_corr = {1: np.float32(2.2143886), 2: np.float32(1.2195894), 3: np.float32(1.4773585), 4: np.float32(2.7058163), 5: np.float32(1.5874418), 6: np.float32(1.8118497), 7: np.float32(1.9383032), 8: np.float32(1.6891266), 9: np.float32(3.5851347), 10: np.float32(1.0379335), 11: np.float32(0.8733048), 12: np.float32(1.0344352), 13: np.float32(2.825319)}
    # amp_corr = {2: np.float32(1.4903895), 3: np.float32(2.0388417), 4: np.float32(3.0054529), 5: np.float32(2.386002), 6: np.float32(2.7193341), 7: np.float32(3.4876633), 8: np.float32(2.3009765), 9: np.float32(3.1773841), 10: np.float32(1.1456816), 11: np.float32(0.9827003), 12: np.float32(1.2655742)}

    for n_freq in n_freqs:
        fft_amp[n_freq] = -1j * 0.5 * symbol_samples * channel_weight[n_freq]
    wave = np.real(np.fft.irfft(fft_amp))
    # adjust the amplitude
    # if np.max(np.abs(wave)) > 0:
    #     wave = wave / np.max(np.abs(wave))
    return np.concatenate([wave, wave])

def plot_symbol(n_freqs):
    signal = gen_symbol(n_freqs)
    plot_audio(signal, rate)

def encode_symbol_to_waveform(symbols):
    signal = np.zeros(0)
    for symbol in symbols:
        signal = np.concatenate([signal, gen_symbol(symbol)])
    return signal

def decode_one_symbol(signal):
    start = 0 + symbol_samples//2
    end = start + symbol_samples
    trunc_signal = signal[start:end]
    fft_result = np.fft.rfft(trunc_signal)
    amp = [np.abs(fft_result[channel]) for channel in range(0, 24)]
    max_amp = np.max(amp)

    # return max
    # max_amp_index = np.argmax([np.abs(fft_result[channel]) for channel in channels])
    # max_channel = channels[max_amp_index]
    # return [max_channel], amp

    # return combination
    result = []
    for channel in channels:
        if np.abs(fft_result[channel]) > 0.13:
            result.append(channel)
    return result, amp

def decode_waveform_to_symbol(signal):
    symbols = []
    for i in range(0, len(signal), 2 * symbol_samples):
        if i + 2 * symbol_samples > len(signal):
            break
        symbol = decode_one_symbol(signal[i:i+2*symbol_samples])
        symbols.append(symbol)
    return symbols

# correction frequency response
for _ in range(5):
    # channel weight correction
    symbols = [channels, channels, channels, channels, channels, channels]
    signal = encode_symbol_to_waveform(symbols)
    record_signal, rate, raw = play_and_record_precise2(signal, rate)
    recv_symbols = decode_waveform_to_symbol(record_signal)

    recv_fft_value = recv_symbols[2][1]

    channel_fft_value = {channel: np.abs(recv_fft_value[channel]) for channel in channels}
    # plt.plot(channels, [channel_fft_value[channel] for channel in channels])

    correction_weight = {channel: 1 / channel_fft_value[channel] for channel in channels}

    channel_weight = {channel: channel_weight[channel] * correction_weight[channel] for channel in channels}
    channel_weight = {channel: channel_weight[channel] / sum(channel_weight.values()) for channel in channels}

max corr 0.17159609310810608
max corr 0.29535146345364804
59805 60376
max corr 0.1671735084356574
max corr 0.2008423868138149
59862 60430
max corr 0.15259669291903352
max corr 0.16178322217460034
59612 60179
max corr 0.17986021948299774
max corr 0.32282376424116616
59811 60382
max corr 0.1560998885962287
max corr 0.18212071180101183
59824 60391


In [137]:
symbols = [[], [], [], channels, [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], []]
signal = encode_symbol_to_waveform(symbols)
record_signal, rate, raw = play_and_record_precise2(signal, rate)
recv_symbols = decode_waveform_to_symbol(record_signal)[2:]
for symbol in recv_symbols:
    plt.plot(symbol[1])
plt.legend([f"Symbol {i, symbols[i]}" for i in range(len(recv_symbols))])
plt.axvline(x=channels[0], color='r')
plt.axvline(x=channels[-1], color='r')

max corr 0.17498520367235523
max corr 0.2789810788055441
59776 61309


qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x63cdae73e550


In [181]:
bin_size = 20 * len(channels)
bin = np.random.randint(0, 2, bin_size)

symbols = []
for i in range(0, len(bin), len(channels)):
    symbol = []
    for j in range(len(channels)):
        if bin[i+j] == 1:
            symbol.append(channels[j])
    symbols.append(symbol)

signal = encode_symbol_to_waveform(symbols)

signal_time = len(signal) / rate
bps = len(bin) / signal_time
print(f"bps: {bps}")

bps: 5500.0


In [179]:
# Transmit

print("number of channels:", len(channels))


similarities = []
def run():
    global rate
    bin_size = 20 * len(channels)
    bin = np.random.randint(0, 2, bin_size)

    symbols = []
    for i in range(0, len(bin), len(channels)):
        symbol = []
        for j in range(len(channels)):
            if bin[i+j] == 1:
                symbol.append(channels[j])
        symbols.append(symbol)

    signal = encode_symbol_to_waveform(symbols + [[], []])
    record_signal, rate, raw = play_and_record_precise2(signal, rate)
    recv_symbols = decode_waveform_to_symbol(record_signal)

    recv_symbols = recv_symbols[:len(symbols)]
    recv_bin = np.random.randint(0, 2, bin_size)
    for i in range(len(recv_symbols)):
        for j in range(len(channels)):
            if channels[j] in recv_symbols[i][0]:
                recv_bin[i*len(channels)+j] = 1
            else:
                recv_bin[i*len(channels)+j] = 0
    similarity = np.sum(recv_bin == bin) / bin_size
    # print(recv_bin == bin)
    # print("similarity:", similarity)
    similarities.append(similarity)

print("start")
for _ in range(100):
    run()
print("end")
# print mean, std, ...
print("mean:", np.mean(similarities))
print("std:", np.std(similarities))
print("max:", np.max(similarities))
print("min:", np.min(similarities))
print("median:", np.median(similarities))
print("percentile 25:", np.percentile(similarities, 25))
print("percentile 75:", np.percentile(similarities, 75))
print("percentile 90:", np.percentile(similarities, 90))
plt.plot(similarities)
plt.show()

number of channels: 11
start
max corr 0.18462931178712555
max corr 0.27975144826057324
59813 61925
max corr 0.16497854972187287
max corr 0.1369588591497168
59741 61837
max corr 0.13261679850403837
max corr 0.12930280878056916
59708 61827
max corr 0.15092830559762596
max corr 0.2062188014577974
59816 61915
max corr 0.15114277653004982
max corr 0.21069907016906975
59607 61706
max corr 0.18227407394653994
max corr 0.30556518676530886
59814 61924
max corr 0.16657266159769485
max corr 0.2066269443746461
59684 61784
max corr 0.1811536149359762
max corr 0.14400669777487807
59701 61818
max corr 0.1846631949749043
max corr 0.22528967312601958
59760 61860
max corr 0.17772785359252652
max corr 0.22417801435453538
59550 61650
max corr 0.15035947505045585
max corr 0.24961468106671603
59788 61897
max corr 0.1814075392040001
max corr 0.20908657162448838
59627 61725
max corr 0.17513050664216287
max corr 0.12778083937561394
59701 61819
max corr 0.1373458660745458
max corr 0.1842006413073926
59703 61802

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x63cdae9236d0


In [84]:

# symbols = [[1], [2], [3], [4], [5]]
symbols = [[], [channel for channel in channels], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], []]
# symbols = [[1, 2, 3, 4, 5], [1, 2, 3, 4], [1, 2, 3], [1, 2], [1]]
# symbols = [[1, 2, 3, 4, 5], []]
# symbols = [[1, 5], [1]]
# symbols = [[], []]

# iterate through all possible combinations
# symbols = [[]]
# for i in range(1, 13):
#     symbols = symbols + [[i, *symbol] for symbol in symbols]

print(len(symbols))
print(symbols)
signal = encode_symbol_to_waveform(symbols)
record_signal, rate, raw = play_and_record_precise2(signal, rate)
recv_symbols = decode_waveform_to_symbol(record_signal)
# print([a for a, b in recv_symbols])
if len(symbols) != len(recv_symbols):
    print(f"Expected: {len(symbols)}, Received: {len(recv_symbols)}")
for recv_symbol, symbol in zip(recv_symbols, symbols):
    # print(f"Expected: {symbol}, Received: {recv_symbol[0]}")
    if set(symbol) != set(recv_symbol[0]):
        print(f"Expected: {symbol}, Received: {recv_symbol[0]}")

14
[[], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], []]
max corr 0.1640240457203356
max corr 0.2166284948454494
59516 60849
Expected: 14, Received: 13
Expected: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], Received: []
Expected: [2], Received: []
Expected: [3], Received: []
Expected: [4], Received: []
Expected: [5], Received: []
Expected: [6], Received: []
Expected: [7], Received: []
Expected: [8], Received: []
Expected: [9], Received: []
Expected: [10], Received: []
Expected: [11], Received: []
Expected: [12], Received: []
